# Resume Test training for S2AE

In [2]:
import math
import sys
import time
import datetime

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
from scipy import spatial

import torch
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary
from tqdm.auto import tqdm

from data_splitter import DataSplitter
from training_set import TrainingSetLidarSeg
from loss import *
# from model_fcn import ModelFCN
# from model_simple_for_testing import ModelSimpleForTesting
from model_unet import ModelUnet
from model_segnet import ModelSegnet
from model import Model
from sphere import Sphere
from visualize import Visualize
from metrics import *
from average_meter import AverageMeter

    
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
print(f"Initializing CUDA...")
torch.cuda.set_device(0)
torch.backends.cudnn.benchmark = True

print(f"Setting parameters...")
bandwidth = 100
learning_rate = 1e-3
n_epochs = 2
batch_size = 5
num_workers = 32
n_classes = 9

print(f"Initializing data structures...")
net = Model(bandwidth=bandwidth, n_classes=n_classes).cuda()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
criterion = MainLoss()
writer = SummaryWriter()

timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
model_save = f'test_lidarseg_{timestamp}'

print('\n')
print(f'All network instances are initialized.')
print(f'Saving final model to {model_save}.')

Initializing CUDA...
Setting parameters...
Initializing data structures...
[Model] We have [2, 20, 40, 120, 180, 120, 40, 20, 9] features.
[Model] We have [100, 40, 30, 15, 10, 8, 10, 15, 30, 40, 100] bandwidths.


All network instances are initialized.
Saving final model to test_lidarseg_20220423130907.


In [4]:
# export_ds = '/mnt/data/datasets/nuscenes/processed'
export_ds = '/media/scratch/berlukas/nuscenes'

# training
cloud_filename = f"{export_ds}/sem_clouds_100_200.npy"

# testing
dec_input = f"{export_ds}/decoded_input_lidar.npy"
dec_clouds = f"{export_ds}/decoded_lidar.npy"
dec_gt = f"{export_ds}/decoded_gt_lidar.npy"

print(f"Loading clouds from {cloud_filename}.")
cloud_features = np.load(cloud_filename)


sem_cloud_features = np.copy(cloud_features[:, 2, :, :])
cloud_features = cloud_features[:, 0:2, :, :]
print(f"Shape clouds is {cloud_features.shape} and sem clouds is {sem_cloud_features.shape}")

Loading clouds from /media/scratch/berlukas/nuscenes/sem_clouds_100_200.npy.
Shape clouds is (200, 2, 200, 200) and sem clouds is (200, 200, 200)


In [5]:
# Initialize the data loaders
train_set = TrainingSetLidarSeg(cloud_features, sem_cloud_features)
print(f"Total size of the training set: {len(train_set)}")
split = DataSplitter(train_set, False, test_train_split=1.0, val_train_split=0.05, shuffle=True)

# Split the data into train, val and optionally test
train_loader, val_loader, test_loader = split.get_split(
    batch_size=batch_size, num_workers=num_workers)
train_size = split.get_train_size()
val_size = split.get_val_size()
test_size = split.get_test_size()


print("Training size: ", train_size)
print("Validation size: ", val_size)
if test_size == 0:
    print('Test size is 0. Configured for external tests')
else:
    print("Testing size: ", test_size)

Total size of the training set: 200
Training size:  190
Validation size:  10
Test size is 0. Configured for external tests


In [5]:
def adjust_learning_rate_exp(optimizer, epoch_num, lr):
    decay_rate = 0.96
    new_lr = lr * math.pow(decay_rate, epoch_num)
    for param_group in optimizer.param_groups:
        param_group['lr'] = new_lr

    return new_lr

def train_lidarseg(net, criterion, optimizer, writer, epoch, n_iter, loss_, t0):
    net.train()
    for batch_idx, (cloud, lidarseg_gt) in enumerate(train_loader):
        cloud, lidarseg_gt = cloud.cuda().float(), lidarseg_gt.cuda().long()        
        enc_dec_cloud = net(cloud)
        loss = criterion(enc_dec_cloud, lidarseg_gt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()        
        loss_ += float(loss)

        writer.add_scalar('Train/Loss', float(loss), n_iter)
        n_iter += 1

        if batch_idx % 10 == 9:
            t1 = time.time()
            print('[Epoch %d, Batch %4d] loss: %.8f time: %.5f lr: %.3e' %
                  (epoch + 1, batch_idx + 1, loss_ / 10, (t1 - t0) / 60, lr))
            t0 = t1
            loss_ = 0.0
    return n_iter

def validate_lidarseg(net, criterion, optimizer, writer, epoch, n_iter):
    avg_pixel_acc = AverageMeter()
    avg_pixel_acc_per_class = AverageMeter()
    avg_jacc = AverageMeter()
    avg_dice = AverageMeter()
    net.eval()
    with torch.no_grad():            
        for batch_idx, (cloud, lidarseg_gt) in enumerate(val_loader):            
            cloud, lidarseg_gt = cloud.cuda().float(), lidarseg_gt.cuda().long()
            enc_dec_cloud = net(cloud)
                        
            optimizer.zero_grad()
            loss = criterion(enc_dec_cloud, lidarseg_gt)                                                                                        
            writer.add_scalar('Validation/Loss', float(loss), n_iter)                        
            
            pred_segmentation = torch.argmax(enc_dec_cloud, dim=1)
            pixel_acc, pixel_acc_per_class, jacc, dice = eval_metrics(lidarseg_gt, pred_segmentation, num_classes = n_classes)
            avg_pixel_acc.update(pixel_acc)
            avg_pixel_acc_per_class.update(pixel_acc_per_class)
            avg_jacc.update(jacc)
            avg_dice.update(dice)

            n_iter += 1
            
        epoch_p_1 = epoch+1
        writer.add_scalar('Validation/AvgPixelAccuracy', avg_pixel_acc.avg, epoch_p_1)   
        writer.add_scalar('Validation/AvgPixelAccuracyPerClass', avg_pixel_acc_per_class.avg, epoch_p_1)   
        writer.add_scalar('Validation/AvgJaccardIndex', avg_jacc.avg, epoch_p_1)
        writer.add_scalar('Validation/AvgDiceCoefficient', avg_dice.avg, epoch_p_1)  
    return n_iter

def save_checkpoint(net, optimizer, criterion, n_epoch):
    checkpoint_path = f'./checkpoints/{model_save}_{n_epoch}.pth'
    torch.save({
            'epoch': n_epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': criterion,
            }, checkpoint_path)
    print('================================')
    print(f'Saved checkpoint to {checkpoint_path}')
    print('================================')

def test_lidarseg(net, criterion, writer):
    all_input_clouds = [None] * test_size
    all_decoded_clouds = [None] * test_size
    all_gt_clouds = [None] * test_size
    k = 0
    avg_pixel_acc = AverageMeter()
    avg_pixel_acc_per_class = AverageMeter()
    avg_jacc = AverageMeter()
    avg_dice = AverageMeter()
    n_iter = 0
    net.eval()
    with torch.no_grad():            
        for batch_idx, (cloud, lidarseg_gt) in enumerate(test_loader):
            cloud, lidarseg_gt = cloud.cuda().float(), lidarseg_gt.cuda().long()
            enc_dec_cloud = net(cloud)
            
            pred_segmentation = torch.argmax(enc_dec_cloud, dim=1)
            pixel_acc, pixel_acc_per_class, jacc, dice = eval_metrics(lidarseg_gt, pred_segmentation, num_classes = n_classes)
            avg_pixel_acc.update(pixel_acc)
            avg_pixel_acc_per_class.update(pixel_acc_per_class)
            avg_jacc.update(jacc)
            avg_dice.update(dice)
            
            writer.add_scalar('Test/PixelAccuracy', pixel_acc, n_iter)   
            writer.add_scalar('Test/PixelAccuracyPerClass', pixel_acc_per_class, n_iter)   
            writer.add_scalar('Test/JaccardIndex', jacc, n_iter)
            writer.add_scalar('Test/DiceCoefficient', dice, n_iter)  
            
            n_batch = enc_dec_cloud.shape[0]
            for i in range(0, n_batch):                
                all_input_clouds[k] = cloud.cpu().data.numpy()[i,:,:,:]
                all_decoded_clouds[k] = enc_dec_cloud.cpu().data.numpy()[i,:,:,:]
                all_gt_clouds[k] = lidarseg_gt.cpu().data.numpy()[i,:,:]
                k = k + 1     
            n_iter += 1
            
        writer.add_scalar('Test/AvgPixelAccuracy', avg_pixel_acc.avg, n_iter)   
        writer.add_scalar('Test/AvgPixelAccuracyPerClass', avg_pixel_acc_per_class.avg, n_iter)   
        writer.add_scalar('Test/AvgJaccardIndex', avg_jacc.avg, n_iter)
        writer.add_scalar('Test/AvgDiceCoefficient', avg_dice.avg, n_iter)  

    return all_input_clouds, all_decoded_clouds, all_gt_clouds

In [6]:
abort = False
train_iter = 0
val_iter = 0
loss_ = 0.0

chkp = './checkpoints/test_lidarseg_20220423102145.pkl_1.pth'

print(f'Loading checkpoint from {chkp}...')
checkpoint = torch.load(chkp)

print('Loading trained model weights...')
net.load_state_dict(checkpoint['model_state_dict'])

print('Loading trained optimizer...')
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

print('Loading trained model loss function...')
criterion = checkpoint['loss']
prev_epochs = checkpoint['epoch'] + 1
print(f"Previously trained for {prev_epochs} epochs...")

# This is very important otherwise we might run in to a CUDA OOM error.
del checkpoint
torch.cuda.empty_cache()

print(f'Starting training using {n_epochs} more epochs')
for epoch in tqdm(range(n_epochs)):    
    lr = adjust_learning_rate_exp(optimizer, epoch_num=epoch, lr=learning_rate)
    t0 = time.time()

    cur_epoch = epoch + prev_epochs
    train_iter = train_lidarseg(net, criterion, optimizer, writer, cur_epoch, train_iter, loss_, t0)    
    val_iter = validate_lidarseg(net, criterion, optimizer, writer, cur_epoch, val_iter)
    writer.add_scalar('Train/lr', lr, cur_epoch)
    save_checkpoint(net, optimizer, criterion, cur_epoch)
        
print("Training finished!")
final_save_path = f'./{model_save}.pkl'
torch.save(net.state_dict(), final_save_path)
print(f'Saved final weights to {final_save_path}.')

Loading checkpoint from ./checkpoints/test_lidarseg_20220423102145.pkl_1.pth...
Loading trained model weights...
Loading trained optimizer...
Loading trained model loss function...
Previously trained for 2 number of epochs...
Starting training using 2 more epochs


/home/berlukas/workspace/python/pytorch-venv/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


[Epoch 2, Batch   10] loss: 0.54206480 time: 1.46805 lr: 1.000e-03
[Epoch 2, Batch   20] loss: 0.51888722 time: 0.90931 lr: 1.000e-03
[Epoch 2, Batch   30] loss: 0.50916074 time: 0.91189 lr: 1.000e-03
Saved checkpoint to ./checkpoints/test_lidarseg_20220423124412_1.pth
[Epoch 3, Batch   10] loss: 0.47568941 time: 0.92686 lr: 9.600e-04
[Epoch 3, Batch   20] loss: 0.45820813 time: 0.91006 lr: 9.600e-04
[Epoch 3, Batch   30] loss: 0.46434557 time: 0.90382 lr: 9.600e-04
Saved checkpoint to ./checkpoints/test_lidarseg_20220423124412_2.pth

Training finished!
Saved final weights to ./test_lidarseg_20220423124412.pkl.
